In [1]:
# input
tweet = u'#HaceInstantes Abrazo al Congreso organizado por gremios docentes después de la represión policial de ayer. https://t.co/ToyertpI70'

In [2]:
print tweet

#HaceInstantes Abrazo al Congreso organizado por gremios docentes después de la represión policial de ayer. https://t.co/ToyertpI70


In [4]:
import warnings

import pandas as pd
import numpy as np
import pickle
import operator
import re
import gc
import gensim

from tw_dataset.dbmodels import *
import json

In [8]:
def normalize(doc):
    pre_doc = doc
        
    # remover URLs
    pre_doc = re.sub(
        r"https?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+",
        " ", pre_doc)
    
    # minúsculas
    pre_doc = pre_doc.lower()

    # volar acentos
    pre_doc = gensim.utils.deaccent(pre_doc)

    # remove bullshit
    pre_doc = re.sub(r"\@|\'|\"|\\|…|\/|\-|\||\(|\)|\.|\,|\!|\?|\:|\;|“|”|’|—", " ", pre_doc)
    
    # contraer vocales
    for v in 'aeiou':
        pre_doc = re.sub(r"[%s]+" % v, v, pre_doc)    
    
    # normalizar espacio en blanco
    pre_doc = re.sub(r"\s+", " ", pre_doc)
    pre_doc = re.sub(r"(^\s)|(\s$)", "", pre_doc)
    
    return pre_doc

In [16]:
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.data import load
from nltk.stem import SnowballStemmer
from string import punctuation

spanish_tokenizer = load('tokenizers/punkt/spanish.pickle')

# stopwords en español
spanish_stopwords = stopwords.words('spanish')

# spanish stemmer
stemmer = SnowballStemmer('spanish')

# punctuation to remove
non_words = list(punctuation)

# we add spanish punctuation
non_words.extend(['¿', '¡'])
non_words.extend(map(str, range(10)))

stemmer = SnowballStemmer('spanish')


def tokenize(text, stem=True, remove_stopwords=False):
    text = text.lower()
    result = []
    
    for sentence in spanish_tokenizer.tokenize(text):
        # remover puntuación
        text = ''.join([c for c in sentence if c not in non_words])
        
        # tokenize
        tokens = word_tokenize(text)

        if remove_stopwords:
            tokens = [t for t in tokens if t not in spanish_stopwords]

        # stem
        if stem:
            tokens = [stemmer.stem(t) for t in tokens]
        
        # tokens de al menos 2 letras
        tokens = [t for t in tokens if len(t) > 1]
        
        result += tokens
        
    return result

In [14]:
print tweet

#HaceInstantes Abrazo al Congreso organizado por gremios docentes después de la represión policial de ayer. https://t.co/ToyertpI70


In [17]:
tweet = normalize(tweet)
print tweet

#haceinstantes abrazo al congreso organizado por gremios docentes despues de la represion policial de ayer


In [18]:
tweet = tokenize(tweet)
print tweet

[u'haceinst', u'abraz', u'al', u'congres', u'organiz', u'por', u'gremi', u'docent', u'despu', u'de', u'la', u'represion', u'policial', u'de', u'ayer']


In [ ]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [35]:
class get_docs(object):
    def __init__(self, corpus):
        self.corpus = corpus

    def __iter__(self):
        for doc in self.corpus:
            tokens = tokenize(preprocess(doc), remove_stopwords=True)
            yield tokens

In [1]:
from gensim.models.phrases import Phraser, Phrases

In [ ]:
phrases = Phrases(get_docs(corpus), min_count=5)
bigram = Phraser(phrases)
trigram = Phrases(bigram[get_docs(corpus)], min_count=5)

In [41]:
dictionary = gensim.corpora.Dictionary(trigram[get_docs(corpus)])
dictionary.filter_extremes(no_below=3, no_above=0.3, keep_n=None)

2017-04-29 20:49:40,029 : INFO : collecting all words and their counts
2017-04-29 20:49:40,043 : INFO : PROGRESS: at sentence #0, processed 0 words and 0 word types
2017-04-29 20:50:20,050 : INFO : PROGRESS: at sentence #10000, processed 72026 words and 74599 word types
2017-04-29 20:51:00,038 : INFO : PROGRESS: at sentence #20000, processed 143398 words and 134683 word types
2017-04-29 20:51:45,573 : INFO : PROGRESS: at sentence #30000, processed 212618 words and 188898 word types
2017-04-29 20:52:26,366 : INFO : PROGRESS: at sentence #40000, processed 284308 words and 243904 word types
2017-04-29 20:53:08,527 : INFO : PROGRESS: at sentence #50000, processed 355860 words and 296243 word types
2017-04-29 20:53:49,600 : INFO : PROGRESS: at sentence #60000, processed 427084 words and 346451 word types
2017-04-29 20:54:30,899 : INFO : PROGRESS: at sentence #70000, processed 500042 words and 396439 word types
2017-04-29 20:55:15,741 : INFO : PROGRESS: at sentence #80000, processed 571445 w

In [42]:
sorted(dictionary.token2id.items(), key=lambda x:-x[1])[:50]

[(u'carohad', 26200),
 (u'pr_prolejsoy', 26199),
 (u'moratori_fiscal', 26198),
 (u'cifr_desaparec', 26197),
 (u'linchamient', 26196),
 (u'tan', 26195),
 (u'junt', 26194),
 (u'sab_abril', 26193),
 (u'aceleracion', 26192),
 (u'monserrat', 26191),
 (u'pi\u2026', 26190),
 (u'lanch', 26189),
 (u'tecnopolisfederal_mision', 26188),
 (u'juni', 26187),
 (u'agu_contamin', 26186),
 (u'preguntasfrecuent', 26185),
 (u'casacreativadelsur', 26184),
 (u'emerg', 26183),
 (u'arame', 26182),
 (u'accionnotici', 26181),
 (u'jefatur_gabinet', 26180),
 (u'cupon', 26179),
 (u'consecuencias\u201d', 26178),
 (u'radi_univers', 26177),
 (u'convencion', 26176),
 (u'buffy', 26175),
 (u'millone\u2026', 26174),
 (u'zuranog', 26173),
 (u'viernessant', 26172),
 (u'abri_convocatori', 26171),
 (u'adriandelu', 26170),
 (u'automatizacion', 26169),
 (u'gveriti_adn', 26168),
 (u'nqnxev', 26167),
 (u'dema\u2026', 26166),
 (u'convocatori_program', 26165),
 (u'anacastellani', 26164),
 (u'corporacion', 26163),
 (u'fundi', 26162)

In [44]:
dictionary.save("tweets_es.dict")

2017-04-29 21:18:18,118 : INFO : saving Dictionary object under tweets_es.dict, separately None
2017-04-29 21:18:18,160 : INFO : saved tweets_es.dict


In [45]:
bow = [dictionary.doc2bow(doc) for doc in trigram[get_docs(corpus)]]

In [46]:
with open('tweets_es_bow.pickle', 'wb') as f:
    pickle.dump(bow,f)